# LeetCode Solutions
----
## About

The purpose of this notebook is to expand on the thoughtprocess behind a LeetCode solution as to fill in the blanks and make sure that I'm not just regurgitating a solution that I've memorized from the past.

---
---

## 1. Two Sum

Given an array of integers, return **indices** of the two numbers such that they add up to a specific target.

You may assume that each input would have **exactly** one solution, and you may not use the *same* element twice.

### Naive Solution

For this question, a brute force solution would be to loop through the array and in each loop, we would have to take the current index plus one and iterate through the rest of the array. If we find two indices `i` and `j` such that `nums[i] + nums[j] == target`, then we return that pair `i` and `j` in an array.

Since we know that each input would have exactly one solution, then it suffices to return an empty array at the bottom of the function to satisfy the return type.

This solution would be $O(n^2)$ in the worst case and takes $O(1)$ space.

In [2]:
from typing import List


class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        for i in range(len(nums)):
            for j in range(i + 1, len(nums)):
                if nums[i] + nums[j] == target:
                    return [i, j]
        
        return []

### Optimal Solution

An optimal solution for this question would be to cache the seen value and its index in a hashmap as we iterate. This helps because we can have an initial check on every iteration to verify if `target - nums[i]` where `i` is the current index we're iterating on is contained within the cache. If that's true, then we've found the correct pair of indices to return. This works because if we've already cached one addend and we're currently iterating on the other addend, then we have `nums[i] + nums[j] == target` as in the brute force solution.

This solution would be $O(n)$ time complexity in the worst case and takes $O(n)$ space.

In [3]:
from typing import List

class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        cache = {}
        
        for i, val in enumerate(nums):
            if target - val in cache:
                return [cache[target - val], i]
            
            cache[val] = i
        
        return []

### Analysis

The trick in this question is to use a dictionary to cache seen values and their corresponding indices in case we run across an addend that allow some value in the dictionary to add up to the target parameter.

So typically, it may be a good idea to use this technique on problems that require us to find values that sum up to a target value.

---
---

## 200. Number of Islands

Given a 2D grid map of `'1'`s (land) and `'0'`s (water), count the number of islands. An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water.

### Proposed Solution

From the outset, this problem looks like a standard search and modify problem. This means that we're have to implement a depth first search on the "graph" and for each element modify it such that we don't need to come across it again. After we conclude each search, we then have to increment the number of islands that we've visited. Finally, we return that number of islands.

Time complexity for this proposed algorithm would be $O(m * n)$ where $m$ is the number of rows and $n$ is the number of columns.

In [4]:
from typing import List


class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        if not grid:
            return 0
        
        num_islands = 0
        
        for i in range(len(grid)):
            for j in range(len(grid[i])):
                if grid[i][j] == '1':
                    self.dfs(grid, i, j)
                    num_islands += 1
        
        return num_islands
    
    def dfs(self, grid: List[List[int]], i: int, j: int):
        if i < 0 or i >= len(grid) or j < 0 or j >= len(grid[0]) or grid[i][j] != '1':
            return
        
        grid[i][j] = '2'
        
        self.dfs(grid, i + 1, j)
        self.dfs(grid, i - 1, j)
        self.dfs(grid, i, j + 1)
        self.dfs(grid, i, j - 1)


---
---

## 42. Trapping Rain Water

Given $n$ non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it is able to trap after raining.

### Proposed Solution

The idea behind doing this is to stray away from calculating area by height times width. We can instead sum the water amount of each bin.

We're going to have two pointers: one on the left and the other on the right. We also keep track of the maximum values on each edge.

On each iteration, we check which values `nums[left]` and `nums[right]` is less than or equal to the other. If the current value on either direction is greater than the corresponding max value then we replace that value, otherwise, we add the difference between the max value and the current value on that direction and increment or decrement the pointer.

We then return the result.

This solution would take $O(n)$ time and $O(1)$ space.

In [5]:
from typing import List


class Solution:
    def trap(self, height: List[int]) -> int:
        left, right = 0, len(height) - 1
        max_left, max_right = -1, -1
        result = 0
        
        while left <= right:
            if height[left] <= height[right]:
                if max_left < height[left]:
                    max_left = height[left]
                else:
                    result += max_left - height[left]
                
                left += 1
            else:
                if max_right < height[right]:
                    max_right = height[right]
                else:
                    result += max_right - height[right]
                
                right -= 1
        
        return result

---
---

## 1192. Critical Connections in a Network

There are `n` servers numbered from `0` to `n-1` connected by undirected server-to-server `connections` forming a network where `connections[i] = [a, b]` represents a connection between servers `a` and `b`. Any server can reach any other server directly or indirectly through the network.

A *critical connection* is a connection that, if removed, will make some server unable to reach some other server.

Return all critical connections in the network in any order.

### Proposed Solutions

An edge is a critical connection, if and only if it is not in a cycle.

So, if we know how to find cycles, and discard all edges in the cycles, then the remaining connections are a complete collection of critical connections.

**How to find edges in cycles, and remove them**

We will use DFS algorithm to find cycles and decide whether or not an edge is in a cycle.

Define **rank** of a node: The depth of a node during a DFS. Starting node has a rank 0.

Only the nodes on the current DFS path have non-special ranks. In other words, only the nodes that we've started visiting, but haven't finished visiting, have ranks. So `0 <= rank <= n`

**How can rank help us with removing cycles**

Imagine you have a current path of length `k` during a DFS. The nodes on the path has increasing ranks from `0` to `k` and incrementing by `1`. Surprisingly, your next visit finds a node that has a rank of `p` where `0 <= p < k`. Why does this happen? You found a node that is on the current search path. That means you found a cycle.

But only the current level of search knows it finds a cycle. How does the upper level of search know if you backtrack? We make sure of the return value of the DFS: the minimum rank it finds. During a step of search from node `u` to its neighbor `v` if `dfs(v)` returns something smaller than or equal to `rank(u)`, then `u` knows its neighbor `v` helped it to find a cycle back to `u` or to `u`'s ancestor. So `u` knows it should discard the edge `(u, v)` which is in a cycle.

In [6]:
from collections import defaultdict
from typing import List


class Solution:
    def criticalConnections(self, n: int, connections: List[List[int]]) -> List[List[int]]:
        graph = self.make_graph(connections)
        connections = set(map(tuple, (map(sorted, connections))))
        rank = [-2] * n
        
        def dfs(node: int, depth: int):
            if rank[node] >= 0:
                return rank[node]
            
            rank[node] = depth
            min_back_depth = n
            
            for neighbor in graph[node]:
                if rank[neighbor] == depth - 1:
                    continue
                
                back_depth = dfs(neighbor, depth + 1)
                
                if back_depth <= depth:
                    # This means we have a cycle
                    connections.discard(tuple(sorted((node, neighbor))))
                
                min_back_depth = min(min_back_depth, back_depth)

            return min_back_depth
        
        dfs(0, 0)
        
        return list(connections)
    
    def make_graph(self, connections: List[List[int]]):
        graph = defaultdict(list)

        for connection in connections:
            graph[connection[0]].append(connection[1])
            graph[connection[1]].append(connection[0])

        return graph

### Ideal Solution

Most solutions to this problem require the use of Tarjan's strongly connected components algorithm.

The basic idea of the algorithm is this: a DFS search begins from an arbitrary start node (and subsequent depth-first searches are conducted on any nodes that have not yet been found). As usual with DFS, the search visits every node of the graph exactly once, declining to revisit any node that has already been visited. Thus, the collection of search trees is a spanning forest of the graph. The strongly connected components will be recovered as certain subtrees of this forest. The roots of these subtress are called the roots of the strongly connected components. Any node of a strongly connected component might server as a root, if it happens to be the first node of a component that is discovered by search.

---
---

## 146. LRU Cache

Design and implement a data structure for Least Recently Used (LRU) cache. It should support the following operations: get and put.

* `get(key)` - Get the value (will always be positive) of the key if the key exists in the cache, otherwise return -1.

* `put(key, value)` - Set or insert the value if the key is not already present. When the cache reached its capacity, it should invalidate the least recently used item before inserting a new item.

The cache is initialized with a positive capacity.

Follow up:
Could you do both operations in O(1) time complexity?

### Solution

This question is straightforward. We need to keep track of two things. How recent a value was inserted into the cache and storing the actual key and value pair in the cache.

We can obviously use a dictionary to keep track of the key and value pairs.

For the recency dictionary, we can store a timestamp on it that has to be updated every time something is queried or removed.


In [7]:
from collections import OrderedDict


class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.ordered = OrderedDict()
        self.cache = {}

    def get(self, key: int) -> int:
        if key in self.cache:
            self.__update_order(key)
            return self.cache[key]
        
        return -1

    def put(self, key: int, value: int) -> None:
        if self.__at_capacity() and key not in self.cache:
            lru = self.__get_lru()
            self.__delete_lru(lru)
        
        self.__update_order(key)
        self.cache[key] = value
    
    def __update_order(self, key: int):
        if key in self.ordered:
            del self.ordered[key]

        self.ordered[key] = 1

    
    def __get_lru(self) -> int:
        if len(self.ordered) == 0:
            return -1
        
        return self.ordered.popitem(last=False)[0]
    
    def __at_capacity(self):
        return len(self.cache) == self.capacity
    
    def __delete_lru(self, lru: int):
        # already deleted from OrderedDict self.recency in get_lru call
        del self.cache[lru]
                

### Analysis

Our solution here has a get and put function implemented in constant time.

---
---

## 2. Add Two Numbers

You are given two non-empty linked lists representing two non-negative integers. The digits are stored in reverse order and each of their nodes contain a single digit. Add the two numbers and return it as a linked list.

You may assume the two numbers do not contain any leading zero, except the number 0 itself.

### Solution

This question has a simple solution. We iterate over the two linked lists in parallel while either one still exists. We keep track of a total value that also keeps track of a carry variable. We then return the result listnode

In [8]:
from dataclasses import dataclass

In [9]:
@dataclass
class ListNode:
    val: int
    next: "ListNode" = None

In [10]:
class Solution:
    def addTwoNumbers(self, l1: ListNode, l2: ListNode) -> ListNode:
        result = ListNode(val=0)
        runner = result
        carry = 0
        
        while l1 or l2:
            total = carry
            
            if l1:
                total += l1.val
                l1 = l1.next
            
            if l2:
                total += l2.val
                l2 = l2.next
            
            runner.next = ListNode(val=total % 10)
            carry = total // 10
            
            runner = runner.next
        
        if carry > 0:
            runner.next = ListNode(val=carry)
        
        return result.next

---
---

## 5. Longest Palindromic Substring

Given a string **s**, find the longest palindromic substring in **s**. You may assume that the maximum length of **s** is 1000.

### Solution

The idea with this solution is to expand around two indices `(i, i)` and `(i, i + 1)`. For each of the indices, we expand around them for each valid palindrome.

In [11]:
class Solution:
    def __init__(self):
        self.lo = 0
        self.max_len = 0
    
    def longestPalindrome(self, s: str) -> str:
        n = len(s)
        
        if n < 2:
            return s
        
        for i in range(n - 1):
            self.extend(s, i, i)
            self.extend(s, i, i + 1)
        
        return s[self.lo: self.lo + self.max_len]
    
    def extend(self, s: str, i: int, j: int):
        while i >= 0 and j < len(s) and s[i] == s[j]:
            i -= 1
            j += 1
        
        if self.max_len < j - i - 1:
            self.lo = i + 1
            self.max_len = j - i - 1

---
---

## 21. Merge Two Sorted Lists

Merge two sorted linked lists and return it as a new list. The new list should be made by splicing together the nodes of the first two lists.

### Solution

This is an easy problem. If the value of one node is less, then we make them the next node.

In [12]:
from dataclasses import dataclass

In [13]:
@dataclass
class ListNode:
    val: int
    next: "ListNode" = None

In [14]:
class Solution:
    def mergeTwoLists(self, l1: ListNode, l2: ListNode) -> ListNode:
        if not l1:
            return l2
        
        if not l2:
            return l1
        
        if l1.val < l2.val:
            l1.next = self.mergeTwoLists(l1.next, l2)
            return l1
        
        l2.next = self.mergeTwoLists(l1, l2.next)
        return l2

---
---

## 20. Valid Parenthesis

Given a string containing just the characters '(', ')', '{', '}', '[' and ']', determine if the input string is valid.

An input string is valid if:

    Open brackets must be closed by the same type of brackets.
    Open brackets must be closed in the correct order.

Note that an empty string is also considered valid.

### Solution

To solve this problem, we have to keep a stack of characters. If we see an opening character, we can store the equivalent closing character in a stack such that if we come across a closing character as we iterate, we can pop the stack and check if the values are equivalent.

We then check if the stack is empty or not

In [15]:
class Solution:
    def isValid(self, s: str) -> bool:
        stack = []
        for char in s:
            if char == '(':
                stack.append(')')
            elif char == '{':
                stack.append('}')
            elif char == '[':
                stack.append(']')
            elif not stack or stack.pop() != char:
                return False
        
        return not stack

---
---

## 4. Median of Two Sorted Arrays

There are two sorted arrays nums1 and nums2 of size m and n respectively.

Find the median of the two sorted arrays. The overall run time complexity should be O(log (m+n)).

You may assume nums1 and nums2 cannot be both empty.

---
---

## 53. Maximum Subarray

Given an integer array `nums`, find the contiguous subarray (containing at least one number) which has the largest sum and return its sum.

### Solution

Kadane's algorithm is the optimal solution here

In [16]:
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        max_so_far, current_max = nums[0], nums[0]
        
        for i in range(1, len(nums)):
            max_so_far = max(nums[i], nums[i] + max_so_far)
            current_max = max(current_max, max_so_far)
        
        return current_max

---
---

## 994. Rotting Oranges

In a given grid, each cell can have one of three values:

    the value 0 representing an empty cell;
    the value 1 representing a fresh orange;
    the value 2 representing a rotten orange.

Every minute, any fresh orange that is adjacent (4-directionally) to a rotten orange becomes rotten.

Return the minimum number of minutes that must elapse until no cell has a fresh orange.  If this is impossible, return -1 instead.

In [17]:
from collections import deque


class Solution:
    def orangesRotting(self, grid: List[List[int]]) -> int:
        if not grid or len(grid) == 0:
            return 0
        
        rows, cols = len(grid), len(grid[0])
        
        queue = deque()
        
        count_fresh = 0
        
        # Put the position of all rotten oranges in queue
        # Count the number of fresh oranges
        for i in range(rows):
            for j in range(cols):
                if grid[i][j] == 2:
                    queue.append([i, j])
                elif grid[i][j] == 1:
                    count_fresh += 1
        
        if count_fresh == 0:
            return 0
        
        count = 0
        dirs = [
            [1, 0],
            [-1, 0],
            [0, 1],
            [0, -1]
        ]
        
        while queue:
            count += 1
            size = len(queue)
            
            for i in range(size):
                px, py = queue.popleft()
                for dir in dirs:
                    x = px + dir[0]
                    y = py + dir[1]
                    
                    if (
                        x < 0 or 
                        y < 0 or 
                        x >= rows or 
                        y >= cols or 
                        grid[x][y] == 0 or 
                        grid[x][y] == 2
                    ):
                        continue
                    
                    grid[x][y] = 2
                    
                    queue.append([x, y])
                    
                    count_fresh -= 1
        
        return count - 1 if count_fresh == 0 else -1

---
---

## 15. 3Sum

Given an array nums of n integers, are there elements a, b, c in nums such that a + b + c = 0? Find all unique triplets in the array which gives the sum of zero.

In [19]:
class Solution:
    def threeSum(self, nums: List[int]) -> List[List[int]]:
        result = []
        
        nums.sort()
        
        for i in range(len(nums)-2):
            if i == 0 or (i > 0 and nums[i] != nums[i-1]):
                j, k = i + 1, len(nums) - 1
                while j < k:
                    target = nums[j] + nums[k]
                    
                    if target == -nums[i]:
                        result.append([nums[i], nums[j], nums[k]])
                        
                        while j < k and nums[j] == nums[j+1]:
                            j += 1
                        
                        while j < k and nums[k] == nums[k-1]:
                            k -= 1
                        
                        j += 1
                        k -= 1
                    elif target > -nums[i]:
                        k -= 1
                    else:
                        j += 1
        return result

---
---

## 202. Happy Number

Write an algorithm to determine if a number is "happy".

A happy number is a number defined by the following process: Starting with any positive integer, replace the number by the sum of the squares of its digits, and repeat the process until the number equals 1 (where it will stay), or it loops endlessly in a cycle which does not include 1. Those numbers for which this process ends in 1 are happy numbers.

In [20]:
class Solution:
    def isHappy(self, n: int) -> bool:
        seen = set()
        
        while n not in seen:
            seen.add(n)
            n = sum([int(x) ** 2 for x in str(n)])
        
        return n == 1

---
---

## 238. Product of Array Except Self

Given an array nums of n integers where n > 1,  return an array output such that output[i] is equal to the product of all the elements of nums except nums[i].

In [21]:
class Solution:
    def productExceptSelf(self, nums: List[int]) -> List[int]:
        result = [1 for _ in range(len(nums))]
        
        for i in range(1, len(nums)):
            result[i] = result[i-1] * nums[i-1]
        
        right, j = 1, len(nums)-1
        
        while j >= 0:
            result[j] *= right
            right *= nums[j]
            j -= 1
        
        return result

---
---

## 3. Longest Substring Without Repeating Characters

Given a string, find the length of the **longest substring** without repeating characters.

### Solution



In [22]:
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        if not s:
            return 0
        
        m = {}
        j = 0
        max_len = 0
        
        for i, val in enumerate(s):
            if val in m:
                j = max(j, m[val] + 1)
            
            m[val] = i
            max_len = max(max_len, i - j + 1)
        
        return max_len

---
---

## 56. Merge Intervals

Given a collection of intervals, merge all overlapping intervals.

In [31]:
class Solution:
    def merge(self, intervals: List[List[int]]) -> List[List[int]]:
        if not intervals:
            return []
        # sort the array
        # keep a stack of the intervals
        # combine intervals where possible
        intervals = sorted(intervals, key=lambda interval: interval[0])
        result = [intervals[0]]
        
        for i in range(1, len(intervals)):
            c_start, c_end = intervals[i]
            if result[-1][1] < c_start:
                result.append([c_start, c_end])
            elif result[-1][1] < c_end:
                result[-1][1] = c_end
                
        return result

In [32]:
Solution().merge([[1,3],[2,6],[8,10],[15,18]])

[[1, 6], [8, 10], [15, 18]]

In [33]:
Solution().merge([[1,4],[4,5]])

[[1, 5]]

In [34]:
Solution().merge([])

[]

## 121. Best Time to Buy and Sell Stock

Say you have an array for which the ith element is the price of a given stock on day i.

If you were only permitted to complete at most one transaction (i.e., buy one and sell one share of the stock), design an algorithm to find the maximum profit.

Note that you cannot sell a stock before you buy one.

In [42]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        if not prices:
            return 0
        
        min_buy = prices[0]
        max_profit = 0
        
        for i in range(1, len(prices)):
            min_buy = min(min_buy, prices[i])
            max_profit = max(max_profit, prices[i] - min_buy)
        
        return max_profit

In [43]:
assert Solution().maxProfit([7, 1, 5, 3, 6, 4]) == 5

---
---

## 206. Reverse Linked List

Reverse a singly linked list

In [44]:
from dataclasses import dataclass

In [45]:
@dataclass
class ListNode:
    val: int
    next: "ListNode" = None

In [46]:
class Solution:
    def reverseList(self, head: ListNode) -> ListNode:
        prev = None
        curr = head
        
        while curr:
            next = curr.next
            curr.next = prev
            prev = curr
            curr = next
        
        return prev

---
---